# Forecasting Basics

This notebook covers the fundamentals of time series forecasting with GradientCastFM.

**Topics:**
- Single series forecasting
- Understanding frequency codes
- Multi-series forecasting
- Response structure and metadata

In [ ]:
import sys
sys.path.append('..')

from gradientcast import GradientCastFM
from utils.synthetic_data import generate_trend_series, generate_seasonal_series, generate_multivariate_series
import matplotlib.pyplot as plt

# Replace with your API key
GRADIENTCAST_API_KEY = "your-api-key-here"

fm = GradientCastFM(api_key=GRADIENTCAST_API_KEY)

---
## 1. Single Series Forecasting

The simplest use case: forecast a single time series.

In [ ]:
# Generate sample daily sales data
historical_data = generate_trend_series(
    n_points=30,
    trend_type='linear',
    base_value=100,
    trend_strength=2,
    noise_level=0.1
)

# Forecast 7 days ahead
result = fm.forecast(
    input_data=historical_data,
    horizon_len=7,
    freq="D"  # Daily frequency
)

# Access forecast values
forecast = result["default"]  # Single series uses "default" as key
print("Forecast values:", [round(v, 1) for v in forecast])

In [ ]:
# Visualize
plt.figure(figsize=(12, 5))
plt.plot(range(len(historical_data)), historical_data, 'b-o', markersize=4, label='Historical')
plt.plot(range(len(historical_data), len(historical_data) + len(forecast)), forecast, 
         'r--o', markersize=4, label='Forecast')
plt.axvline(x=len(historical_data)-0.5, color='gray', linestyle='--', alpha=0.5)
plt.xlabel('Day')
plt.ylabel('Value')
plt.title('Single Series Forecast')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

---
## 2. Understanding Frequency Codes

The `freq` parameter tells the model about your data's time granularity, which helps it apply appropriate patterns.

| Code | Description | Examples |
|------|-------------|----------|
| `H` | Hourly | Server metrics, energy consumption |
| `D` | Daily | Daily sales, website visits |
| `W` | Weekly | Weekly aggregates |
| `M` | Monthly | Monthly revenue, subscriptions |
| `Q` | Quarterly | Quarterly reports |
| `Y` | Yearly | Annual metrics |

In [ ]:
# Monthly data example
monthly_revenue = generate_seasonal_series(
    n_points=24,  # 2 years of monthly data
    base_value=50000,
    seasonality='monthly',
    amplitude=0.15,
    noise_level=0.05
)

# Forecast 6 months ahead
result = fm.forecast(
    input_data=monthly_revenue,
    horizon_len=6,
    freq="M"  # Monthly frequency
)

forecast = result["default"]

# Visualize
plt.figure(figsize=(12, 5))
months = list(range(len(monthly_revenue) + len(forecast)))
plt.plot(months[:len(monthly_revenue)], monthly_revenue, 'b-o', markersize=4, label='Historical')
plt.plot(months[len(monthly_revenue):], forecast, 'r--o', markersize=4, label='Forecast')
plt.axvline(x=len(monthly_revenue)-0.5, color='gray', linestyle='--', alpha=0.5)
plt.xlabel('Month')
plt.ylabel('Revenue ($)')
plt.title('Monthly Revenue Forecast (freq="M")')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

---
## 3. Multi-Series Forecasting

Forecast multiple related time series in a single API call for better efficiency.

In [ ]:
# Generate correlated time series (e.g., sales in different regions)
multi_data = generate_multivariate_series(
    n_series=3,
    n_points=30,
    base_values=[100, 150, 80],
    correlation=0.6
)

# Rename for clarity
input_data = {
    "region_north": multi_data["series_0"],
    "region_south": multi_data["series_1"],
    "region_west": multi_data["series_2"]
}

# Forecast all series at once
result = fm.forecast(
    input_data=input_data,
    horizon_len=7,
    freq="D"
)

# Access forecasts by name
for region in input_data.keys():
    print(f"{region}: {[round(v, 1) for v in result[region]]}")

In [ ]:
# Visualize all series
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

for ax, (region, historical), color in zip(axes, input_data.items(), colors):
    forecast = result[region]
    ax.plot(range(len(historical)), historical, f'-o', color=color, markersize=3, label='Historical')
    ax.plot(range(len(historical), len(historical) + len(forecast)), forecast, 
            '--o', color=color, markersize=3, alpha=0.7, label='Forecast')
    ax.axvline(x=len(historical)-0.5, color='gray', linestyle='--', alpha=0.3)
    ax.set_title(region)
    ax.set_xlabel('Day')
    ax.legend(fontsize=8)
    ax.grid(alpha=0.3)

plt.suptitle('Multi-Series Forecast', fontsize=12)
plt.tight_layout()
plt.show()

---
## 4. Response Structure

The forecast response contains both predictions and useful metadata.

In [ ]:
# Make a forecast and inspect the response
result = fm.forecast(
    input_data=generate_trend_series(50, noise_level=0.05),
    horizon_len=10,
    freq="H"
)

# Access model metadata
info = result.model_info

print("Response Structure:")
print(f"  - Forecast values: result['default'] or result.forecast['default']")
print(f"  - Model info: result.model_info")
print(f"  - Raw response: result.raw")
print()
print("Model Info:")
print(f"  Context length used: {info.context_length}")
print(f"  Requested horizon: {info.requested_horizon_length}")
print(f"  Frequency: {info.frequency}")
print(f"  Covariates used: {info.used_covariates}")
print(f"  Processing time: {info.processing_time:.3f}s")

In [ ]:
# Convert to DataFrame (requires pandas)
df = result.to_dataframe()
print("DataFrame output:")
print(df.head(10))

---
## Key Takeaways

1. **Single series**: Pass a list directly, results are under `"default"` key
2. **Multi-series**: Pass a dict, results are keyed by your series names
3. **Frequency matters**: Choose the right `freq` for better pattern recognition
4. **Response metadata**: `model_info` provides useful debugging information

**Next:** [Using Covariates](02_covariates.ipynb) to improve forecast accuracy